# Projeto Final

## Reconhecimento de convicção através da fala para suporte ao ensino

Disciplina: Computação Afetiva - IA369Y

Prof. Paula D. P. Costa


Alunos:

    Diego Garrido
    Fernanda Ferreira
    Ricardo Keigo

## Importação de bibliotecas

In [17]:
%matplotlib inline
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn import svm
from sklearn.cross_validation import train_test_split
import glob
from hmmlearn.hmm import GaussianHMM, MultinomialHMM
from sklearn.externals import joblib

## Carregamento dos dados em formato csv

In [18]:
datas_cer = glob.glob('features/treinamento/certeza/*.csv')
datas_inc = glob.glob('features/treinamento/incerteza/*.csv')

## Organizar as características para cada um dos áudios e concatená-los

In [19]:
def select_features(datas):
    feat_list = []
    for file in datas:
        df = pd.read_csv(file, sep=';')
        #df1 = df[df.columns[2:4]]
        df1 = df.loc[:, 'alphaRatio_sma3': 'F3amplitudeLogRelF0_sma3nz'].values
        d = np.array(df1)
        #print (d.shape)
        feat_list.append(d)
    lenghts = []
    for i in range(len(feat_list)):
        lenghts.append(len(feat_list[i]))
        #print (df1)
    f = np.vstack(feat_list)
    return f, lenghts, feat_list

In [20]:
f_cer, len_cer, list_fcer = select_features(datas_cer)
f_inc, len_inc, list_finc = select_features(datas_inc)
#f_control, len_control = select_features(datas_control)

## Treinamento do Classificador HMM

In [21]:
import warnings
warnings.filterwarnings('ignore')

model1 = GaussianHMM(n_components=15, covariance_type='diag', n_iter=50)
model1.fit(f_cer, len_cer)
print ('Dados do treinamento:', model1.monitor_)
print ('Convergência:', model1.monitor_.converged)

Dados do treinamento: 
 ConvergenceMonitor(history=[-1682049.9027937579, -1682049.179727497], iter=50,
          n_iter=50, tol=0.01, verbose=False)
Convergência: True


In [22]:
model2 = GaussianHMM(n_components=15, covariance_type='diag', n_iter=50)
model2.fit(f_inc, len_inc)
print ('Dados do treinamento:', model2.monitor_)
print ('Convergência:', model2.monitor_.converged)

Dados do treinamento: 
 ConvergenceMonitor(history=[-1818662.1069479825, -1818650.887878068], iter=50,
          n_iter=50, tol=0.01, verbose=False)
Convergência: True


## Salvar o modelo treinado em pkl

In [23]:
joblib.dump(model1, "Model1_certeza.pkl")

['Model1_certeza.pkl']

In [24]:
joblib.dump(model2, "Model2_incerteza.pkl")

['Model2_incerteza.pkl']

In [25]:
#Exemplo do Carregamento do modelo.
model1 = joblib.load("Model1_certeza.pkl")
model2 = joblib.load("Model2_incerteza.pkl")

Para testar o modelo, utilize:
    
    prob1 = model1.score(list_vcer[val])
    prob2 = model2.score(list_vcer[val])
    if prob1 > prob2:
        print (['certeza'])
    else:
        print (['incerteza'])

## Validação do Classificador para cada um dos áudios

In [26]:
datas_v_cer = glob.glob('features/validacao/certeza/*.csv')
datas_v_inc = glob.glob('features/validacao/incerteza/*.csv')

v_cer, ven_cer, list_vcer = select_features(datas_v_cer)
v_inc, ven_inc, list_vinc = select_features(datas_v_inc)

### Verificar a quantidade de acerto

In [27]:
# Para os áudios classificados como CERTEZA
def taxa_cer(list_vcer): #taxa de erro e acerto em porcentagem
    erros1 = []
    acertos1 = []
    for val in range(len(list_vcer)):
        prob1 = model1.score(list_vcer[val])
        prob2 = model2.score(list_vcer[val])
        if prob1 > prob2:
            acertos1.append(['certeza'])
        else:
            erros1.append(['incerteza'])
    taxa_ac1 = (len(acertos1))/len(list_vcer)*100
    taxa_er1 = (len(erros1))/len(list_vcer)*100
    print ('taxa de acerto:', taxa_ac1, 'taxa de erro:', taxa_er1)
    return taxa_ac1, taxa_er1

# Para os áudios classificados como INCERTEZA
def taxa_inc(list_vinc): #taxa de erro e acerto em porcentagem
    erros2 = []
    acertos2 = []
    for val2 in range(len(list_vinc)):
        prob3 = model1.score(list_vinc[val2])
        prob4 = model2.score(list_vinc[val2])
        if prob3 > prob4:
            erros2.append(['certeza'])
        else:
            acertos2.append(['incerteza'])
    taxa_ac2 = (len(acertos2))/len(list_vinc)*100
    taxa_er2 = (len(erros2))/len(list_vinc)*100
    print ('taxa de acerto:', taxa_ac2, 'taxa de erro:', taxa_er2)
    return taxa_ac2, taxa_er2


print ('Validação para certeza')
ac1, er1 = taxa_cer(list_vcer)

print ('Validação para incerteza')
ac2, er2 = taxa_inc(list_vinc)

Validação para certeza
taxa de acerto: 63.26530612244898 taxa de erro: 36.734693877551024
Validação para incerteza
taxa de acerto: 57.14285714285714 taxa de erro: 42.857142857142854


## Extra

### Audio de teste

In [28]:
def select_feature_test(path=''):
    df = pd.read_csv(path, sep=';')
    df1 = df.loc[:, 'alphaRatio_sma3': 'F3amplitudeLogRelF0_sma3nz'].values
    d = np.array(df1)
    #print (d.shape)
    lenghts = []
    for i in range(len(d)):
        lenghts.append(len(d[i]))
    #print (df1)
    f = np.vstack(d)
    return f, lenghts

In [29]:
path_t = 'features_projeto_IA369Y/certeza/I1questao13.csv'
#path_t2 = 'features_projeto_IA369Y/incerteza/I21questao11.csv'

f_test, len_test = select_feature_test(path_t)


# Teste das perguntas de controle
prob1 = model1.score(f_test)
prob2 = model2.score(f_test)

if prob1 > prob2:
    print ('certeza')
else:
    print ('incerteza')

certeza


## Referências

- https://hmmlearn.readthedocs.io/en/latest/index.html

- https://github.com/tiagoft/course_audio

- https://github.com/naxingyu/opensmile